In [13]:
import torch
import numpy as np
from torch import nn, optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [14]:
class CauchyActivation(nn.Module):
  def __init__(self):
    super().__init__()

    # Initializing λ1, λ2, d as trainable parameters
    self.lambda1 = nn.Parameter(torch.tensor(1.0))
    self.lambda2 = nn.Parameter(torch.tensor(1.0))
    self.d = nn.Parameter(torch.tensor(1.0))

  def forward(self, x):
    x2_d2 = x ** 2 + self.d ** 2
    return self.lambda1 * x / x2_d2 + self.lambda2 / x2_d2

In [20]:
class MyDataset(Dataset):
    def __init__(self, x):
        self.x = x
        self.y = x/(x**2+9)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [21]:
x = torch.arange(-100, 100, 0.001)
dataset = MyDataset(x)
train_data, test_data = torch.utils.data.random_split(dataset, [0.8, 0.2])
train_dataloader = DataLoader(
            train_data,
            batch_size=64,
            shuffle=True,
)
test_dataloader = DataLoader(
            test_data,
            batch_size=64,
            shuffle=True,
)

In [22]:
model = CauchyActivation()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
epochs=10
print("λ1 = {} λ2 = {} d = {}".format(model.lambda1.item(), model.lambda2.item(), model.d.item()))
for t in range(epochs):
    print(f"Epoch {t+1}\n....")
    model.train()
    for step, (data,label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        if step % 1000 ==0:
            loss, current = loss.item(), step+1
            print(f"loss:{loss:>7f}, step:{current:>5d}\n")
    print("λ1 = {} λ2 = {} d = {}".format(model.lambda1.item(), model.lambda2.item(), model.d.item()))

λ1 = 1.0 λ2 = 1.0 d = 1.0
Epoch 1
....
loss:0.009494, step:    1

loss:0.001909, step: 1001

loss:0.000688, step: 2001

λ1 = 0.6608152389526367 λ2 = 0.15149952471256256 d = 1.5314451456069946
Epoch 2
....
loss:0.000174, step:    1

loss:0.000142, step: 1001

loss:0.000128, step: 2001

λ1 = 0.7572121620178223 λ2 = 0.004013596568256617 d = 1.8414981365203857
Epoch 3
....
loss:0.000148, step:    1

loss:0.000057, step: 1001

loss:0.000038, step: 2001

λ1 = 0.870467483997345 λ2 = 6.91399909555912e-05 d = 2.35874080657959
Epoch 4
....
loss:0.000055, step:    1

loss:0.000016, step: 1001

loss:0.000005, step: 2001

λ1 = 0.9711529612541199 λ2 = 0.0018921648152172565 d = 2.842089891433716
Epoch 5
....
loss:0.000002, step:    1

loss:0.000000, step: 1001

loss:0.000000, step: 2001

λ1 = 0.9995215535163879 λ2 = -0.00010395119898021221 d = 2.9973886013031006
Epoch 6
....
loss:0.000000, step:    1

loss:0.000000, step: 1001

loss:0.000000, step: 2001

λ1 = 0.9999980926513672 λ2 = 8.297581644001184

In [ ]:
model.eval()
output=model(x)
print(x.shape, output.shape)
plt.plot(x.numpy(),output.detach().numpy(),'-kx')

torch.Size([200000]) torch.Size([200000])
